# Proceso  de EDA para estudiar los datos, y preparacion para hacer un modelo de recomendacion

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import gzip
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
import pickle



## Steam Games

In [2]:
steam_games = pd.read_csv('./datasets/steam_games.csv')

In [ ]:
steam_games.info()

In [ ]:
fig,ax = plt.subplots(1,3)
sns.boxplot(data=steam_games,y='price',ax = ax[0])
sns.boxplot(data=steam_games,y='price',ax = ax[1])
sns.boxplot(data=steam_games,y='price',ax = ax[2])
ax[0].set_ylim(0,100)
ax[1].set_ylim(0,400)
ax[2].set_ylim(0,800)
plt.show()

### En el grafico se puede ver que la columna price tiene muchos valores atipicos que superan los 20 y llegan a los 1000, sin embargo en una app de juegos es posible tener tal distribucion, es decir muchos juegos de precio bajo y varios de precio alto por esto no se hara un tratado especial

In [ ]:
sns.histplot(data=steam_games,x='Year',hue='Free to Play',bins=10,palette='rainbow',multiple='dodge')

plt.ylim(0,20000)
plt.show()

### 2015 es el año con mas juegos y mas juegos Free To Play

## Users Reviews

In [ ]:
user_review = pd.read_csv('./datasets/user_reviews.csv')

In [ ]:
user_review.sample(5)

In [ ]:
sns.histplot(data=user_review,x='sentiment_analysis',bins=10,palette='rainbow',hue='recommend')

plt.show()

### Como se ve del total de sentimiento malos, un alto porcentaje son recomendaciones falsas, y de los buenos el total de recomendaciones true son altas, Para ambas hay un porcentaje existente de recomendaciones que parecen opuesta al sentimiento

## Users Items

In [ ]:
with gzip.open('datasets/user_items_proc.csv.gz', 'rb') as f:
    user_items_proc = pd.read_csv(f, encoding='utf-8')

In [ ]:
user_items_proc.sample(5)

In [ ]:
user_items_proc['item_id'].nunique()

In [ ]:
user_items_proc.describe()

In [ ]:
sns.scatterplot(data=user_items_proc,x='items_count',y='playtime_forever')
plt.show()

# Modelo de recomendacion

In [ ]:

variables = steam_games.drop(columns=['app_name','release_date','specs','price','id','Year','Accounting','developer']).columns
developer_dummies = pd.get_dummies(steam_games['developer'], prefix='developer')
data_encoded = pd.concat([steam_games, developer_dummies], axis=1)
X = data_encoded[list(variables)]
similarities = cosine_similarity(X, X)
knn = NearestNeighbors(n_neighbors=5, metric='cosine', algorithm='brute', n_jobs=-1)
knn.fit(X)
with open('./model/recommend-item-item.pkl', 'wb') as archivo_pkl:
    pickle.dump(knn, archivo_pkl)

In [ ]:
def recomendacion_jueg_fuc(id:int):
    steam_games = pd.read_csv('./datasets/steam_games.csv')
    with open('./model/recommend-item-item.pkl', 'rb') as archivo_pkl:
        knload = pickle.load(archivo_pkl)
    if id not in list(steam_games['id']):
        return "Ese id no pertence a ningun item"
    game_index = steam_games[steam_games['id'] == 761140].index[0]
    distances, indices = knn.kneighbors([X.iloc[game_index]])
    juegos_recomendados = {}

    for i in range(len(indices[0])):
        clave = f"juego {i + 1}" 
        valor = steam_games.iloc[indices[0][i]]['app_name'] 
        juegos_recomendados[clave] = valor 
    return juegos_recomendados